# <div align="center"> CV Api Test </div>

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from k12libs.utils.nb_easy import k12ai_post_request
from k12libs.utils.nb_easy import k12ai_post_cv_request
from k12libs.utils.nb_easy import k12ai_get_data
from k12libs.utils.nb_easy import k12ai_del_data
from k12libs.utils.nb_easy import k12ai_out_data
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_count_down
from k12libs.utils.nb_widget import k12ai_schema_parse

import json

In [ ]:
schema_mnist = k12ai_post_request(
    uri='k12ai/framework/schema',
    data={
        'service_name': 'k12cv',
        'service_task': 'cls',
        'dataset_name': 'cifar10',
    }
)
schema_json = json.loads(schema_mnist)['data']
# k12ai_print(schema_json)
print(schema_json)

In [ ]:
generator = k12ai_schema_parse(json.loads(schema_json), lan='cn', debug=False);

In [ ]:
context = k12ai_post_cv_request(
    uri='k12ai/framework/execute',
    op='train.stop',
    user='10000001',
    uuid='10000001',
)
k12ai_del_data(context['key'])

In [ ]:
context = k12ai_post_cv_request(
    uri='k12ai/framework/execute',
    op='train.start',
    user='10000001',
    uuid='10000001',
    params=generator.get_all_kv()
)
k12ai_del_data(context['key'])

In [ ]:
k12ai_out_data(context['key'], ['status', 'metrics', 'error'], hook_func=k12ai_count_down(10))